# Terminology mapping

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
def runQuery(endpointUrl, query):
    sparql = SPARQLWrapper(endpointUrl)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [3]:
repository = "http://sparql_open:7200/repositories/data"

queryLocalTerms = """PREFIX roo: <http://www.cancerdata.org/roo/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?type ?value
from <http://data.local/rdf>
WHERE {
	?obj rdf:type ?type.
    ?obj roo:local_value ?value .
    FILTER(?type NOT IN (owl:NamedIndividual)).
}
ORDER BY ?type"""

# get local values
localValues = runQuery(repository, queryLocalTerms)
for localValue in localValues["results"]["bindings"]:
    print(localValue["type"]["value"] + ": " + localValue["value"]["value"])

http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28421: 0
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28421: 1
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C48884: 2
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C48884: 1
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C48884: 0
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C48885: 4
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C48885: 2
http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C48885: 3


In [4]:
import rdflib
g = rdflib.Graph()
g.load("https://raw.githubusercontent.com/RadiationOncologyOntology/ROO/master/owl/ROO.owl")

INFO:rdflib:RDFLib Version: 4.2.1


In [22]:
for localValue in localValues["results"]["bindings"]:
    myQuery = "SELECT ?label WHERE { <%s> rdfs:label ?label. }" % (localValue["type"]["value"])
    for row in g.query(myQuery):
        print(row.label + ": " + localValue["value"]["value"])

print({'One': 1, 'Two': 2, 'Three': 3})

Sex: 0
Sex: 1
Generic Regional Lymph Nodes TNM Finding: 2
Generic Regional Lymph Nodes TNM Finding: 1
Generic Regional Lymph Nodes TNM Finding: 0
Generic Primary Tumor TNM Finding: 4
Generic Primary Tumor TNM Finding: 2
Generic Primary Tumor TNM Finding: 3
{'Three': 3, 'Two': 2, 'One': 1}


In [38]:
import ipywidgets as widgets
myWidgets = [ ]

for localValue in localValues["results"]["bindings"]:
    superClass = localValue["type"]["value"]
    localValue = localValue["value"]["value"]
    
    myQuery = "SELECT ?label WHERE { <%s> rdfs:label ?label. }" % superClass
    superClassName = ""
    for row in g.query(myQuery):
        superClassName = row.label
    
    nameWidget = widgets.HTML(value=("<p>%s</p>" % superClassName))
    valueWidget = widgets.Text(value=localValue, disabled=True)
    
    myQuery = """
        SELECT ?subClass ?label 
        WHERE { 
            ?subClass rdfs:subClassOf* <%s>.
            ?subClass rdfs:label ?label.
            
            FILTER (?subClass != <%s>).
        }
        ORDER BY ?label""" % (superClass, superClass)
    
    queryResult = g.query(myQuery)
    classList = list()
    i=0
    for subClassResultRow in queryResult:
        classList.append({"uri": str(subClassResultRow.subClass), "label": str(subClassResultRow.label), "id": i})
        i = i+1
    
    dropdownValues = dict()
    for myClass in classList:
        dropdownValues[myClass["label"]] = myClass["id"]
    standardValWidget = widgets.Dropdown(
        options=dropdownValues
    )
    
    myWidgets.append(widgets.HBox([nameWidget, valueWidget, standardValWidget]))
widgets.VBox(myWidgets)